<a href="https://colab.research.google.com/github/Isafon/ECE528/blob/main/ECE528_ASN1_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Keras

In [ ]:
import numpy as np
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
from keras import datasets, layers, models
import matplotlib.pyplot as plt

### Download and prepare the CIFAR10 dataset


The CIFAR10 dataset contains 60,000 color images in 10 classes, with 6,000 images in each class. The dataset is divided into 50,000 training images and 10,000 testing images. The classes are mutually exclusive and there is no overlap between them.

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

### Verify the data

To verify that the dataset looks correct, let's plot the first 25 images from the training set and display the class name below each image.


In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    # The CIFAR labels happen to be arrays,
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

### Create the model (your code goes here)

In [ ]:
# Importing again because the imports at the beginning weren't working for me
import tensorflow as tf
from tensorflow.keras import layers, models

CIFAR10_MEAN = [0.4914, 0.4822, 0.4465]
CIFAR10_STD  = [0.2470, 0.2435, 0.2616]

norm = layers.Normalization(mean=CIFAR10_MEAN, variance=[s**2 for s in CIFAR10_STD])

data_aug = tf.keras.Sequential([
    layers.ZeroPadding2D(4),
    layers.RandomCrop(32, 32),
    layers.RandomFlip("horizontal"),
], name="data_aug")

def conv_block(x, f, reps=2, pool=True, drop=0.25):
    for _ in range(reps):
        x = layers.Conv2D(f, 3, padding="same", use_bias=False)(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
    if pool:
        x = layers.MaxPool2D()(x)
    if drop:
        x = layers.Dropout(drop)(x)
    return x

inputs = layers.Input((32, 32, 3))
x = norm(inputs)
x = data_aug(x)

x = conv_block(x, 32,  reps=2, pool=True,  drop=0.20)
x = conv_block(x, 64,  reps=2, pool=True,  drop=0.25)
x = conv_block(x, 128, reps=2, pool=True,  drop=0.30)

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10)(x)

model = models.Model(inputs, outputs, name="cifar10_cnn_plus")
print("Params:", model.count_params())

### Compile and train the model

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10,
                    validation_data=(test_images, test_labels))

### Evaluate the model

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
print(test_acc)